In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 16.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_11139/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-29 01:32:06.335234: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:32:06.367445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 01:32:06.367479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 01:32:06.368717: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 01:32:06.374532: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:32:06.375427: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[30, 15], [45, 30], [60, 30]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02, 0.1])

Testando combinacao1: Hidden Size=[30, 15], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  5.861593882999841
Stopped at epoch:  58
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  2.3164153719999376
Stopped at epoch:  50
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.45642219700039277
Stopped at epoch:  9
1/1 [==============================] - 0s 16ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.3858948639999653
Stopped at epoch:  9
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.47660337500019523
Stopped at epoch:  12
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.913924532999772
Stopped at epoch:  21
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 7 +++++

2024-02-29 01:32:36.101323: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  1.5430519019992062
Stopped at epoch:  13
1/1 [==============================] - 0s 66ms/step
+++++++++++ [1_0] | 24 ++++++++++++++++++
Elapsed time:  0.48998464599935687
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [1_0] | 25 ++++++++++++++++++
Elapsed time:  1.221989055999984
Stopped at epoch:  15
1/1 [==============================] - 0s 22ms/step
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  0.34660181300023396
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.6694213780001519
Stopped at epoch:  13
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.36363605899987306
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.30550700499952654
Stopped at epoch:  6
1/1 [==============================] - 0s 32ms/st

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_3_8_0,0.803477,0.697890,0.965653,0.864675,0.036765,0.052849,0.010997,0.033154
model_3_8_1,0.808515,0.703690,0.959211,0.862905,0.035822,0.051835,0.013060,0.033587
model_3_8_2,0.812749,0.706539,0.952274,0.859716,0.035031,0.051336,0.015281,0.034369
model_3_8_3,0.816247,0.706661,0.944907,0.855232,0.034376,0.051315,0.017639,0.035467
model_3_8_4,0.819045,0.704351,0.937190,0.849612,0.033853,0.051719,0.020110,0.036844
model_3_8_5,0.821290,0.700046,0.929270,0.843114,0.033433,0.052472,0.022646,0.038436
model_3_8_6,0.823037,0.694149,0.921163,0.835900,0.033106,0.053503,0.025242,0.040204
model_3_8_20,0.823983,0.551084,0.815702,0.716959,0.032929,0.078530,0.059007,0.069343
model_3_8_7,0.824419,0.687043,0.913040,0.828217,0.032847,0.054747,0.027842,0.042086
model_3_8_19,0.824701,0.562539,0.822300,0.725347,0.032795,0.076526,0.056895,0.067288


DataFrame salvo em ../results/metrics_3_8
+++++++++++ [3_9] | 1 ++++++++++++++++++
Elapsed time:  0.48913711399927706
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_9] | 2 ++++++++++++++++++
Elapsed time:  0.3383981919996586
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [3_9] | 3 ++++++++++++++++++
Elapsed time:  0.35181894400011515
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_9] | 4 ++++++++++++++++++
Elapsed time:  0.3814587910001137
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [3_9] | 5 ++++++++++++++++++
Elapsed time:  0.37296983799933514
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [3_9] | 6 ++++++++++++++++++
Elapsed time:  0.3544772470004318
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [3_9] | 7 ++++++++++++++++++
Elapsed time:  0.296778926

...

DataFrame salvo em ../results/metrics_12_7
+++++++++++ [12_8] | 1 ++++++++++++++++++
Elapsed time:  0.37778402700041624
Stopped at epoch:  6
1/1 [==============================] - 0s 34ms/step
+++++++++++ [12_8] | 2 ++++++++++++++++++
Elapsed time:  0.3057553029993869
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_8] | 3 ++++++++++++++++++
Elapsed time:  0.26971332199991593
Stopped at epoch:  7
1/1 [==============================] - 0s 16ms/step
+++++++++++ [12_8] | 4 ++++++++++++++++++
Elapsed time:  0.33997247600018454
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [12_8] | 5 ++++++++++++++++++
Elapsed time:  0.275303339999482
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [12_8] | 6 ++++++++++++++++++
Elapsed time:  0.28236722300061956
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [12_8] | 7 ++++++++++++++++++
Elapsed time:  0.3

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_8_24,0.897282,0.568082,-0.458159,0.327948,0.019216,0.050468,0.077501,0.063190
model_12_8_23,0.898803,0.573476,-0.433004,0.338188,0.018932,0.049838,0.076164,0.062227
model_12_8_22,0.900478,0.579414,-0.405144,0.349507,0.018618,0.049144,0.074683,0.061163
model_12_8_21,0.902309,0.585916,-0.374514,0.361932,0.018276,0.048384,0.073055,0.059994
model_12_8_20,0.904344,0.593032,-0.340015,0.375791,0.017895,0.047553,0.071222,0.058691
model_12_8_19,0.906580,0.600820,-0.301769,0.391088,0.017477,0.046643,0.069189,0.057253
model_12_8_18,0.909026,0.609297,-0.259481,0.407915,0.017019,0.045652,0.066941,0.055671
model_12_8_17,0.911716,0.618508,-0.212292,0.426527,0.016516,0.044576,0.064433,0.053921
model_12_8_16,0.914703,0.628466,-0.158743,0.447323,0.015957,0.043413,0.061587,0.051965
model_12_8_15,0.917965,0.639143,-0.099142,0.470202,0.015347,0.042165,0.058419,0.049814


DataFrame salvo em ../results/metrics_12_8
+++++++++++ [12_9] | 1 ++++++++++++++++++
Elapsed time:  0.3708479830002034
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [12_9] | 2 ++++++++++++++++++
Elapsed time:  0.26178803400034667
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [12_9] | 3 ++++++++++++++++++
Elapsed time:  0.2804245830002401
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [12_9] | 4 ++++++++++++++++++
Elapsed time:  0.2614768439998443
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [12_9] | 5 ++++++++++++++++++
Elapsed time:  0.2403237429998626
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [12_9] | 6 ++++++++++++++++++
Elapsed time:  0.37826004799990187
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [12_9] | 7 ++++++++++++++++++
Elapsed time:  0.35

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_12_9_24,0.896994,0.733991,0.416552,0.593137,0.019270,0.037208,0.092578,0.063265
model_12_9_23,0.897775,0.734993,0.422677,0.596556,0.019124,0.037068,0.091606,0.062733
model_12_9_22,0.898618,0.736097,0.429371,0.600297,0.018966,0.036914,0.090544,0.062151
model_12_9_21,0.899547,0.737318,0.436848,0.604469,0.018793,0.036743,0.089357,0.061502
model_12_9_20,0.900565,0.738622,0.445224,0.609112,0.018602,0.036561,0.088028,0.060781
model_12_9_19,0.901667,0.740093,0.454433,0.614234,0.018396,0.036355,0.086567,0.059984
model_12_9_18,0.902878,0.741706,0.464765,0.619963,0.018169,0.036129,0.084928,0.059093
model_12_9_17,0.904173,0.743469,0.476092,0.626243,0.017927,0.035883,0.083130,0.058117
model_12_9_16,0.905569,0.745395,0.488652,0.633191,0.017666,0.035613,0.081137,0.057036
model_12_9_15,0.907068,0.747470,0.502616,0.640885,0.017386,0.035323,0.078922,0.055840


DataFrame salvo em ../results/metrics_12_9
